In [1]:
import numpy as np
from numpy import array as arr

import pap   # für Tests

# Entwicklung interner Funktionen für pap.py

In [2]:
ZIFFERN = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')

## Diverses

### _negativ_wird_null

In [3]:
'''
Benötigt von:
* pap.resultat()


Benutzt:
* numpy as np
'''


def _negativ_wird_null(array):
    '''
    "Rampenfunktion": Die Werte von array, die negativ sind, werden durch 0 ersetzt.
    
    
    Argument
    --------
    array : np.ndarray (number_like)
    '''
    
    
    nullen = np.full(np.shape(array), 0)
    return np.where(array > 0, array, nullen)

### _istbool 

In [4]:
'''
Benötigt von:
* pap.resultat()
'''


def _istbool(wert, bool_wert:bool):
    '''
    Überprüft ob wert wirklich bool_wert (True oder False) ist und nicht nur 1 oder 0.
    '''
    
    return (wert == bool_wert and isinstance(wert, bool))    

### _listen_transponieren 

In [5]:
'''
Benötigt von:
* pap.vergleichstabelle()
'''


def _listen_transponieren(listen_matrix):
    '''
    Transponiert eine Liste, die eine Form wie ein 2D np.ndarray hat.
    
    
    Argumente
    ---------
    listen_matrix : list
        Muss aus einer Liste von N Listen, welche alle M Elemente haben, bestehen.
        Also "shape"(listen_matrix) = (N, M).
    
    
    Output
    ------
    tranxponierte_matrix : list
        "shape"(transponierte_maxtrix) = (M, N)
    '''
    
    
    
    transponierte_matrix = []
    for i in range(len(listen_matrix[0])):
        zeile = []
        for j in range(len(listen_matrix)):
            zeile.append(listen_matrix[j][i])
        transponierte_matrix.append(zeile)
    
    return transponierte_matrix

In [6]:
# Test: _listen_transponieren()

test_listen_matrix = [['1', '2'], ['3', '4'], ['5', '6']]
print(_listen_transponieren(test_listen_matrix))

[['1', '3', '5'], ['2', '4', '6']]


## Rundung u.Ä.

### _größenordnung

In [7]:
'''
Benötigt von:
* pap.vergleichstabelle()
* pap._nachkommastelle()
* pap._erste_ziffer()


Benötigt:
* numpy als np
* np.array() als arr()
'''


def _größenordnung(zahlen, art = int):
    '''
    Bestimmt elementweise die Größenordnungen eines Arrays von Zahlen und gibt sie als int- oder float-Array 
    zurück. 
    0 bekommt die Größenordnung 0.
    
    
    Argumente
    ---------
    zahlen : np.ndarray (number_like)
        
    art : type, optional
        Darf nur sein:
        int   - größenordnungen-Array wird aus np.int64-Zahlen bestehen, zB. 0.048 -> -2
        
        float - größenordnungen-Array wird aus np.float64-Zahlen bestehen. zB. 0.048 -> -2.0
        
        
    Output
    ------
    größenordnungen : np.ndarray  (np.int64 oder np.float64)
        np.shape(größenordnungen) = np.shape(zahlen)
    '''
    
    
    
    if art == int:
        art = 'int64'
    elif art == float:
        art = 'float64'
    
    # Nullen werden kompatibel gemacht.
    einsen            = np.ones(np.shape(zahlen))   # 1 hat Größenordnung 0 (welches auch 0 hier haben soll).
    zahlen_kompatibel = np.where(zahlen == 0, einsen, zahlen)
    
    return arr(np.floor(np.log10(np.abs(zahlen_kompatibel))), dtype = art)

### _erste_ziffer 

In [8]:
'''
Benötigt von:
* pap._nachkommastelle()


Benötigt:
* numpy als np

* pap._größenorndung()
'''


def _erste_ziffer(zahlen, art = float):
    '''
    Bestimmt elementweise die erste Ziffer eines Arrays von Zahlen wenn art = int, aber
    wenn art = float, wird die Zahl nur auf Größenordnung null gebracht.
    
    
    Argumente
    ---------
    zahlen : np.ndarray (number_like)
    
    art : type, optional
        Darf nur sein:
        int   - ziffern-Array wird aus np.int64-Zahlen bestehen, zB. 239.78 -> 2
        
        float - ziffern-Array wird aus np.float64-Zahlen bestehen, zB. 239.78 -> 2.3978 
        
    
    Output
    ------
    ziffern : np.ndarray  (np.int64 oder np.float64)
        np.shape(ziffern) = np.shape(zahlen)
    '''
    
    
    
    größenordnungen = _größenordnung(zahlen, art = float)
    ziffern         = np.abs(zahlen / 10**größenordnungen)
    
    if art == float:
        return ziffern
    if art == int:
        return np.int_(ziffern)

In [9]:
# Test: _erste_ziffer()

print(_erste_ziffer(arr([1.293, 37, 0.9304, -1.60, -0.999999, 0.4, 0.6, -0.4, -0.6])))     
print(_erste_ziffer(arr([1.293, 37, 0.9304, -1.60, -0.999999, 0.4, 0.6, -0.4, -0.6]), art = int))

[1.293   3.7     9.304   1.6     9.99999 4.      6.      4.      6.     ]
[1 3 9 1 9 4 5 4 5]


### _nachkommastelle

In [10]:
'''
Benötigt von:
* pap.vergleichstabelle()


Benötigt:
* numpy als np

* _größenordnung
* _erste_ziffer
'''


def _nachkommastelle(zahlen, sig_stellen = 1, sig_grenze = 1.0):
    '''
    Berechnet elementweise die Anzahl der Nachkommastellen, auf die eine Zahl gerundet werden soll, 
    abhängig davon wie viele signifikante Stellen zugelassen werden und davon wo die Signifikanz-Grenze liegt.
    Mehr Details unter "Berechnung".
    
    
    Argumente
    ---------
    zahlen : np.ndarray (number_like)
    
    sig_stellen : int, optional
        Nur Zahlen >= 1 sinnvoll
    
    sig_grenze : int, float, optional
        Nur Zahlen >= 1 und < 10 sinvoll
    
    
    Output
    ------
    nachkommastellen : np.ndarray (np.float64)
        np.shape(nachkommastellen) = np.shape(zahlen)
    
    
    Berechnung
    ----------
    Rundet man eine Zahl wissenschaftlich, dann will man sie auf eine bestimmte Anzahl signifikanter Stellen 
    runden, zB. bei Fehlerwerten auf oft nur eine: 1.389342 -> 1
    Stur auf eine signifikante Stelle zu runden wird aber bei Zahlen mit kleinen ersten Ziffern problematisch
    wie in oberem Beispiel. Der gerundete Wert könnte zwischen 0.5 und 1.5 liegen, wodurch man einen unnötig 
    großen Darstellungsfehler introduziert.
    Um dieses Problem zu beheben, wurde eine Signifikanzgrenze eingeführt. Ist die erste Ziffer des zu 
    rundenden Wertes kleiner als diese Grenze, wird der Wert mit einer signifikanten Stelle mehr gerundet.
    
    Beispiel 1: Sei die Signifikanzgrenze sig_grenze = 4 (Darstellungsfehler max. 1/8)
                und die normale Anzahl signifikanter Stellen sig_stellen = 1
    0.002458 -> 0.0025 (nachkommastelle =  4)
    828.003  -> 800    (nachkommastelle = -2)
    4.001    -> 4      (nachkommastelle =  0)
    3.999    -> 4.0    (nachkommastelle =  1)
    3.949    -> 3.9    (nachkommastelle =  1)
    1.034    -> 1.0    (nachkommastelle =  1)
    0.928    -> 0.9    (nachkommastelle =  1)
    0.434    -> 0.4    (nachkommastelle =  1)
    0.394    -> 0.39   (nachkommastelle =  2)
    
    Beispiel 2: Will man bei oberen Beispielen vermeiden, dass sowohl 4 als auch 4.0 auftauchen,
    kann man sig_grenze = 3.95 wählen. Dann ergibt sich
    4.000 -> 4
    3.999 -> 4
    3.950 -> 4
    3.949 -> 3.9
    '''
    
    
    über_der_grenze      = _erste_ziffer(zahlen) >= sig_grenze
    sig_stellen_normal   = np.full(np.shape(zahlen), sig_stellen)
    signifikante_stellen = np.where(über_der_grenze, sig_stellen_normal, sig_stellen_normal + 1)
    größenordnungen      = _größenordnung(zahlen, art = float)
    
    nachkommastellen     = -größenordnungen + signifikante_stellen - 1
    return nachkommastellen

In [11]:
# Test: _nachkommastelle()
## Demonstration, wie sich _nachkommastelle() auf die rundung diverser Zahlen auswirkt.

zahlen = arr([[7.394, 4.000, 3.950, 3.949, 1.000],
              [0.949, 0.400, 0.395, 0.394, 0.100]])

for zahlliste in zahlen:
    zahlliste = arr(zahlliste)
    nachkommastellen   = _nachkommastelle(zahlliste, sig_grenze = 3.95)
    
    zahlliste_gerundet = pap._rundung(zahlliste, np.int_(nachkommastellen))
    anzahl_stellen = _negativ_wird_null(np.int_(nachkommastellen))
    strings_gerundet = ['{:.{prec}f}'.format(zahlliste_gerundet[i], prec = stelle)
                        for i, stelle in enumerate(anzahl_stellen)]
    
    print(zahlliste)
    print(_größenordnung(zahlliste))
    print(nachkommastellen)    
    print(strings_gerundet)
    print('')

[7.394 4.    3.95  3.949 1.   ]
[0 0 0 0 0]
[0. 0. 0. 1. 1.]
['7', '4', '4', '3.9', '1.0']

[0.949 0.4   0.395 0.394 0.1  ]
[-1 -1 -1 -1 -1]
[1. 1. 1. 2. 2.]
['0.9', '0.4', '0.4', '0.39', '0.10']



## String-Formatierung 

### _füllen

In [12]:
'''
Benötigt von:
* pap.vergleichstabelle()
* pap._zahlen_ausrichtung()
'''


def _füllen(string, index = 0,  menge = 1, füllzeichen = ' '):
    '''
    fügt eine bestimmte Menge an Füllzeichen an eine bestimmte Position eines Strings.
    Ähnlich zu einem .format()-Feature wo der String eine festgelegte Breite haben soll und mit entsprechend
    vielen Füllzeichen aufgefüllt wird. Hier kann man stattdessen die Anzahl der Füllzeichen festlegen und auch
    die genaue Einsetz-Position.
    
    
    Argumente
    ---------
    string : str
    
    index : int, str, optional
        Position im string, wo die füllzeichen eingefügt werden sollen
        Wenn type(index) = str, dann darf index nur sein:
        index = 'links'    (= 0) -> füllzeichen ganz links einfügen
        
        index = 'rechts'   (= len(string)) -> füllzeichen ganz rechts einfügen
    
    menge : int, optional
        Anzahl gewünschter füllzeichen
    
    füllzeichen : str, optional
    
    
    Output
    ------
    string_neu : str
    '''
    
    
    if index == 'rechts':
        index = len(string)
    elif index == 'links':
        index = 0
        
    string_neu = string[:index] + füllzeichen * menge + string[index:]
    return string_neu

In [13]:
# Test: _füllen()

print(repr(_füllen('hi', index = 1, menge = 2)))
print(repr(_füllen('hi', index = len('hi'), menge = 2)))
print(repr(_füllen('hi', index = 'rechts', menge = 2)))

'h  i'
'hi  '
'hi  '


### _entfernen 

In [14]:
'''
Benötigt von:
* pap._zahlen_ausrichtung
'''

def _entfernen(string, index, länge = 1):
    '''
    entfernt eine bestimmte Anzahl (länge) Zeichen von einem String ab einem bestimmten Index.
    
    
    Argumente
    ---------
    string : str
    
    index : int, optional
        Nur index >= 0 sinnvoll.

    länge : int, optional
        Nur länge >= 0 sinnvoll.
        
    
    Output
    ------
    string_output : str
    '''
    
    
    return string[:index] + string[(index + länge):]

In [15]:
# Test: _entfernen()

print(_entfernen('Hi !', 2))
print(_entfernen('Hi !', 2, 3))
print(_entfernen('Hi !', 0, 3))

## Absurde Eingaben
#print(_entfernen('Hi !', 2, -2))
#print(_entfernen('Hi !', -1, 1))

Hi!
Hi
!


### _zentrieren_rechts

In [16]:
'''
Benötigt von:
* pap.vergleichstabelle()
* pap._zahlen_ausrichtung()

Benötigt:
* numpy als np
'''


def _zentrieren_rechts(string, länge_gewünscht, füllzeichen = ' '):
    '''
    arbeitet so wie '{:^{width}'.format(string, width = länge_gewünscht), mit dem Unterschied dass .format bei 
    einer ungeraden Anzahl Füllzeichen, den string ein bisschen nach links anordnet:    'Hi' -> ' Hi  '
    Diese Funktion ordnet in einem solchen Fall den string ein bisschen nach rechts an: 'Hi' -> '  Hi '
    Ist die Anzahl Füllzeichen gerade, benehmen sich beide Funktionen gleich:           'Hi' -> '  Hi  '
    
    Argumente
    ---------
    string : str
    
    länge_gewunscht : int
    
    füllzeichen : str, optional
    
    
    Output
    ------
    string_zentriert : str
    '''
    
    
    länge_string = len(string)
    if länge_gewünscht <= länge_string:
        return string
    else:
        füll_länge        = länge_gewünscht - länge_string
        füll_länge_links  = int(np.ceil(füll_länge / 2))
        füll_länge_rechts = int(np.floor(füll_länge / 2))
        string_zentriert  = füllzeichen * füll_länge_links + string + füllzeichen * füll_länge_rechts
        return string_zentriert

In [17]:
# Test: _zentrieren_rechts()

print(repr(_zentrieren_rechts('Hi', 6)))
print(repr(_zentrieren_rechts('Hi', 5)))
print(repr(_zentrieren_rechts('Hi', 4)))
print(repr(_zentrieren_rechts('Hi', 3)))
print(repr(_zentrieren_rechts('Hi', 2)))
print(repr(_zentrieren_rechts('Hi', 1)))
print(repr(_zentrieren_rechts('Hi', 0)))
print(repr(_zentrieren_rechts('Hi', -1)))
print(repr(_zentrieren_rechts('Hi', 5, '-')))

'  Hi  '
'  Hi '
' Hi '
' Hi'
'Hi'
'Hi'
'Hi'
'Hi'
'--Hi-'


### _zahlen_ausrichtung 

In [18]:
'''
Benötigt von:
* pap.vergleichstabelle()


Benötigt:
* numpy als np

* pap._füllen()
* pap._entferne()
* pap._zentrieren_rechts()
'''


def _zahlen_ausrichtung(zahlen_liste):
    '''
    wandelt eine Liste unterschiedlich langer Zahlenstrings in eine Lister gleichlanger Zahlenstrings um,
    die nach Komma ('.'), Größenordnung ('e') und Einheit (zB. ' %') ausgerichtet sind, siehe "Beispiel".
    
    
    Argumente
    ---------
    zahlen_liste : list  (str)
        Unterstützte Arten von Strings sind:
        * int- und float-Strings (auch mit 'e')
        * solche strings mit einem Leerzeichen und Einheit hinten dran (zB. '1.4e-3 %')
        * einzelne Zeichen (zB. '-')
        * leere Strings
        
    
    Output
    ------
    zahlen_liste : list  (str)
        
    
    Beispiel
    --------
    >>> pap._zahlenausrichtung(['23.3',
    ...                         '2.940e12',
    ...                         '-94',
    ...                         '2030 %'
    ...                         '-6.19e-3'
    ...                         '-',
    ...                         ''])
    ['   23.3       ',
     '    2.940e12  ',
     '  -94         ',
     '-2030        %',
     '   -6.19 e-3  ',
     '      -       ',
     '              ']
    '''
    
    
    # Vorbearbeitung der Liste
    anzahl_strings    = len(zahlen_liste)
    if anzahl_strings == 0:
        return zahlen_liste
    
    ''' Nur Zahlenstrings können ausgerichtet werden, deshalb werden alle anderen
    herausgeplückt.'''
    test_zahl    = [any(x in string for x in ZIFFERN) 
                    for string in zahlen_liste]
    nicht_zahlen = [(i, zahlen_liste.pop(i))  for i in reversed(range(anzahl_strings))  
                    if not test_zahl[i]] [::-1]
    ''' Pflückt alle nicht-Zahlen von hinten nach vorne aus der zahlen_liste um deren richtige Indices zu 
    bewahren.'''
    
    
    # Ausrichten der Zahlenstrings
    if len(zahlen_liste) != 0:

        '''Die Zahlenstrings werden in 9 Bereiche unterteilt:
        minus, vor_komma, komma, nach_komma, e, e-, nach_e, text und ende.
        String-Bsp: '-23.383e-10 %' '''
        anzahl_strings       = len(zahlen_liste)
        anzahl_bereiche = 9 
        indices         = np.int_(np.zeros((anzahl_strings, anzahl_bereiche)))
        längen_strings  = [len(string) for string in zahlen_liste]
        

        '''Nach allen diesen Zeichen unten werden die Strings ausgerichtet. Insbesondere sollen '.' und 'e' 
        alle vertikal übereinander sfein.'''
        test_minus   = ['-'  in string[0] for string in zahlen_liste]
        test_komma   = ['.'  in string    for string in zahlen_liste]
        test_e       = ['e'  in string    for string in zahlen_liste]
        test_e_minus = ['e-' in string    for string in zahlen_liste]
        test_text    = [' '  in string    for string in zahlen_liste]
        
        
        '''Hier werden die Anfangs-Indices aller 9 Bereiche bestimmt. Ist ein Bereich nicht vorhanden, bekommt 
        er den Index des nachfolgenden Bereiches. Deshalb werden die Indices vom hintersten Bereich vorwärts
        bestimmt.'''
        for i in range(anzahl_strings):
            indices[i][-1] = längen_strings[i]   # ende
            if test_text[i]:
                indices[i][7] = zahlen_liste[i].find(' ')
            else:
                indices[i][7] = indices[i][-1]
            if test_e[i]:
                indices[i][4] = zahlen_liste[i].find('e')
                if test_e_minus[i]:
                    indices[i][5] = zahlen_liste[i].find('e-') + 1
                    indices[i][6] = indices[i][5] + 1
                else: 
                    indices[i][5:7] = [indices[i][4] + 1] * 2
            else:
                indices[i][4:7] = [indices[i][7]] * 3
            if test_komma[i]:
                indices[i][2] = zahlen_liste[i].find('.')
                indices[i][3] = indices[i][2] + 1
            else:
                indices[i][2:4] = [indices[i][4]] * 2
            if test_minus[i]:
                indices[i][1] = 1
            else:
                indices[i][1] = 0

        
        '''Aus allen Indices werden die Längen der einzelnen Bereiche bestimmt und daraus die Menge an
        Füllung, die benötigt wird. Diese wird schließlich bei allen Strings an der richtigen Stelle#
        eingefügt.'''
        längen      = (indices.T[1:] - indices.T[:-1]).T   
            # Längen der Bereiche, sortiert nach Bereich, np.shape(längen) = (anzahl_bereiche, anzahl_strings)
        längen_max  = np.max(längen, axis = 0)   # Maximum-Länge jedes Bereiches
        längen_diff = längen_max[np.newaxis:] - längen     
            # Anzahl benötigte Leerzeichen um jeden Bereich bis zur Max-Länge aufzufüllen.

        ## Einfügen der Füllungen
        längen_auswahl  = [7, 6, 5, 4, 3, 2, 1, 0]
        indices_auswahl = [7, 5, 5, 4, 4, 2, 0, 0]   
        '''Dies ist wie folgt zu lesen: Bereich text (7) wird als erstes aufgefüllt. Bereiche e- (5) und 
        nach-e (6) bleiben zusammen und es wird der Platz 5 zwischen dem 'e' und dem '-' aufgefüllt. 
        Auch der Platz 4 zwischen Bereichen e (4) und nach-komma (3) wird gefüllt. Das gleiche gilt für
        Platz 0 vor Bereichen minus (0), vor-komma (1) und komma (2). Fehlt im Zahlenstring irgendeiner
        dieser Bereiche wird sein Platz ebenso gefüllt.'''
        for i in range(anzahl_strings):
            for j in range(len(längen_auswahl)):
                zahlen_liste[i] = _füllen(zahlen_liste[i], menge = längen_diff[i][längen_auswahl[j]], 
                                          index = indices[i][indices_auswahl[j]])
    
        
        # Nachbearbeitung bei Sonderfällen
        '''Wenn Minuszeichen vorhanden sind, wird überall ein Leerzeichen an Füllung deswegen eingebracht.
        Da die Minuszeichen aber bei ihren Zahlen bleiben, kann es passieren, dass eine ganze Spalte an 
        Leerzeichen existiert (all(...)). Dieser wird hier entfernt, da überflüssig.'''
        test_leerzeichen = [' ' in string[0] for string in zahlen_liste]
        test_e_leer      = ['e ' in string if 'e' in string else True for string in zahlen_liste]
        if all(test_leerzeichen):
            for i in range(anzahl_strings):
                zahlen_liste[i] = zahlen_liste[i][1:]
        if all(test_e_leer) and any(test_e):
            index_e_string = test_e.index(True)
            index_nach_e   = zahlen_liste[index_e_string].index('e') + 1
            zahlen_liste   = [_entfernen(string, index_nach_e) for string in zahlen_liste]
        
        
        länge_gesamt = len(zahlen_liste[0])
    else:   # Falls keine Zahlenstrings in zahlen_liste vorhanden
        länge_gesamt = max([len(string) for i, string in nicht_zahlen])
    
    
    # Zentrierte nicht-Zahlenstrings werden wieder in die Liste eingefügt.
    for index, string in nicht_zahlen:
        zahlen_liste.insert(index, _zentrieren_rechts(string, länge_gesamt))
            
            
    return zahlen_liste

In [19]:
liste = ['1.003', '789e-3', '-', '', ' ', '-29.4834e100', '-0.000334', 'x', '53449', '-- %', '27 %', 
         '-5.81 %', '1.239e-24 %']
liste_ausgerichtet = _zahlen_ausrichtung(liste)
for string in liste_ausgerichtet:
    print(repr(string))

'    1.003         '
'  789       e -3  '
'         -        '
'                  '
'                  '
'  -29.4834  e100  '
'   -0.000334      '
'         x        '
'53449             '
'       -- %       '
'   27            %'
'   -5.81         %'
'    1.239   e-24 %'


In [20]:
liste = ['-', '', ' ', 'x', '-- %']
liste_ausgerichtet = _zahlen_ausrichtung(liste)
for string in liste_ausgerichtet:
    print(repr(string))

'  - '
'    '
'    '
'  x '
'-- %'


In [21]:
liste = ['-']
liste_ausgerichtet = _zahlen_ausrichtung(liste)
for string in liste_ausgerichtet:
    print(repr(string))

'-'


In [22]:
liste = ['1.003', '789e-3', '-29.4834e100', '-0.000334', '53449', '27 %', '-5.81 %', '1.239e-24 %']
liste_ausgerichtet = _zahlen_ausrichtung(liste)
for string in liste_ausgerichtet:
    print(repr(string))

'    1.003         '
'  789       e -3  '
'  -29.4834  e100  '
'   -0.000334      '
'53449             '
'   27            %'
'   -5.81         %'
'    1.239   e-24 %'


In [23]:
liste = ['-29.4834e100']
liste_ausgerichtet = _zahlen_ausrichtung(liste)
for string in liste_ausgerichtet:
    print(repr(string))

'-29.4834e100'


In [24]:
# Test - leere Liste

liste = []
liste_ausgerichtet = _zahlen_ausrichtung(liste)
for string in liste_ausgerichtet:
    print(repr(string))

### _plus_minus

In [25]:
'''
Benötigt von:
* pap.vergleichstabelle()
'''


def _plus_minus(string_liste):
    '''entscheidet, welche Strings in einer Liste von Strings mit einem Plus-Minus-Zeichen versehen werden 
    sollen und welche nicht. Diese Funktion ergibt nur als direkte Hilfsfunktion von pap.vergleichstabelle()
    Sinn.
    
    
    Argumente
    ---------
    string_liste : list  (str) 
    
    
    Output
    ------
    plus_minus : list  (str)
    '''
    
    
    plus_minus = []
    for string in string_liste:
        if string == '':   # Falls string leer ist.
            plus_minus.append('')
        elif all([zeichen == ' ' for zeichen in string]):   # Falls string nur aus Leerzeichen besteht.
            plus_minus.append('   ')
        else:
            plus_minus.append(' ± ')   # Falls string aus mehr besteht.
    
    return plus_minus

In [26]:
liste = ['', '    ', '123']
_plus_minus(liste)

['', '   ', ' ± ']